In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
annotations_path = []
for dirname, _, filenames in os.walk('/kaggle/input/face-mask-detection/annotations'):
    for filename in filenames:
        annotations_path.append(os.path.join(dirname, filename))
images_path = []
for dirname, _, filenames in os.walk('/kaggle/input/face-mask-detection/images'):
    for filename in filenames:
        images_path.append(os.path.join(dirname, filename))
        
annotations_path.sort()
images_path.sort()

import torch
print(torch.cuda.memory_summary(device=None, abbreviated=False))
#torch.cuda.empty_cache()

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def get_equation_line_y(tangent, constant, x):
        return tangent*x +constant
    
def get_equation_line_x(tangent, constant, y):
    return round((y+constant)/tangent)

def create_mask(xmin, xmax, ymin, ymax, i, masks):
    two_third_y = (ymax - ymin)*2/3 + ymin
    third_x = (xmax - xmin)/3 + xmin
    two_third_x = (xmax-xmin)*2/3 + xmin
    tangent_left = (two_third_y - ymax)/(xmin - third_x)
    constant_left = two_third_y - tangent_left*xmin
    tangent_right = -tangent_left
    constant_right = two_third_y - tangent_right*xmax
    
    '''for y in range(ymin, ymax):
        if y<two_third_y:
            masks[xmin:xmax,y, i] = 1
        else:
            for x in range(xmin, xmax-1):
                if y<get_equation_line_y(tangent_left, constant_left, x) and y<get_equation_line_y(tangent_right, constant_right, x):
                        masks[x,y,i] = 1
    '''
    for y in range(ymin, ymax):
        if y<two_third_y:
            masks[xmin-1,y, i] = 1
            masks[xmax-2,y, i] = 1
        else:
            for x in range(get_equation_line_x(tangent_left, -constant_left, y)-1,
                           get_equation_line_x(tangent_right, -constant_right, y)-2):
                masks[x,y,i] = 1
    return masks

In [ ]:
import torchvision
from torchvision import transforms, datasets, models
import torch
from PIL import Image
import xml.etree.ElementTree as et

class MaskDataset(object):
    def __init__(self, transforms):
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir("/kaggle/input/face-mask-detection/images/")))
    
    def __getitem__ (self, idx):
        img_path = os.path.join('/kaggle/input/face-mask-detection/images/maksssksksss' + str(idx) + '.png')
        annotation_path = os.path.join("/kaggle/input/face-mask-detection/annotations/maksssksksss" + str(idx) + ".xml")
        img = Image.open(img_path).convert("RGB")
        target = {}
        with open(annotation_path) as annotation:
            xml = et.parse(annotation)
            root = xml.getroot()
            bounding_boxes = []
            labels = []
            for i in range(4,len(root)):
                #bounding_boxes.append([root[i][5][0].text,
                #                           root[i][5][1].text, root[i][5][2].text, root[i][5][3]])
                bounding_boxes.append([int(root[i][5][0].text),
                                           int(root[i][5][1].text), int(root[i][5][2].text), int(root[i][5][3].text)])
                if root[i][0].text== 'with_mask':
                    labels.append(1)
                elif root[i][0].text=='without_mask':
                    labels.append(2)
                else:
                    labels.append(3)
                    
            masks = np.zeros([int(root[2][0].text), int(root[2][1].text), len(bounding_boxes)], dtype='uint8')
            for i in range(len(bounding_boxes)):
                box = bounding_boxes[i]
                col_s, col_e = box[1], box[3]
                row_s, row_e = box[0], box[2]
                masks = create_mask(row_s, row_e, col_s, col_e, i, masks)
            
            
            boxes = torch.as_tensor(bounding_boxes, dtype=torch.float32)
            
            labels = torch.as_tensor(labels, dtype=torch.int64)
            
            img_id = torch.tensor([idx])
            
            masks = torch.as_tensor(masks, dtype=torch.int8)
            
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = img_id
            
        if self.transforms is not None:
            img = self.transforms(img)
        
        return img, target
    
    def __len__(self):
        return len(self.imgs)
    

In [ ]:
import random

data_transform = transforms.Compose([
        transforms.ToTensor(), 
])

def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = []
valid_dataset = []
dataset = MaskDataset(data_transform)
for i in range(len(dataset)):
    if i % 5 == 0:
        valid_dataset.append(dataset[i])
    else:
        train_dataset.append(dataset[i])

train_data_loader = torch.utils.data.DataLoader(
  train_dataset, batch_size=4, collate_fn=collate_fn)

valid_data_loader = torch.utils.data.DataLoader(
  valid_dataset, batch_size=4, collate_fn=collate_fn)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
for imgs, annotations in train_data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    for i in range(len(imgs)):
        print(len(annotations[i]["masks"]))
        print(len(annotations[i]["masks"][0]))
        print(len(annotations[i]["masks"][0][0]))
    break

In [ ]:
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load an instance segmentation model pre-trained pre-trained on COCO
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 4)

# now get the number of input features for the mask classifier
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
# and replace the mask predictor with a new one
model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       4)

In [ ]:
#forrás : https://github.com/matterport/Mask_RCNN

from collections import defaultdict, deque
import datetime
import pickle
import time

import torch
import torch.distributed as dist

import errno
import os


class SmoothedValue(object):
    """Track a series of values and provide access to smoothed values over a
    window or the global series average.
    """

    def __init__(self, window_size=20, fmt=None):
        if fmt is None:
            fmt = "{median:.4f} ({global_avg:.4f})"
        self.deque = deque(maxlen=window_size)
        self.total = 0.0
        self.count = 0
        self.fmt = fmt

    def update(self, value, n=1):
        self.deque.append(value)
        self.count += n
        self.total += value * n

    def synchronize_between_processes(self):
        """
        Warning: does not synchronize the deque!
        """
        if not is_dist_avail_and_initialized():
            return
        t = torch.tensor([self.count, self.total], dtype=torch.float64, device='cuda')
        dist.barrier()
        dist.all_reduce(t)
        t = t.tolist()
        self.count = int(t[0])
        self.total = t[1]

    @property
    def median(self):
        d = torch.tensor(list(self.deque))
        return d.median().item()

    @property
    def avg(self):
        d = torch.tensor(list(self.deque), dtype=torch.float32)
        return d.mean().item()

    @property
    def global_avg(self):
        return self.total / self.count

    @property
    def max(self):
        return max(self.deque)

    @property
    def value(self):
        return self.deque[-1]

    def __str__(self):
        return self.fmt.format(
            median=self.median,
            avg=self.avg,
            global_avg=self.global_avg,
            max=self.max,
            value=self.value)


def all_gather(data):
    """
    Run all_gather on arbitrary picklable data (not necessarily tensors)
    Args:
        data: any picklable object
    Returns:
        list[data]: list of data gathered from each rank
    """
    world_size = get_world_size()
    if world_size == 1:
        return [data]

    # serialized to a Tensor
    buffer = pickle.dumps(data)
    storage = torch.ByteStorage.from_buffer(buffer)
    tensor = torch.ByteTensor(storage).to("cuda")

    # obtain Tensor size of each rank
    local_size = torch.tensor([tensor.numel()], device="cuda")
    size_list = [torch.tensor([0], device="cuda") for _ in range(world_size)]
    dist.all_gather(size_list, local_size)
    size_list = [int(size.item()) for size in size_list]
    max_size = max(size_list)

    # receiving Tensor from all ranks
    # we pad the tensor because torch all_gather does not support
    # gathering tensors of different shapes
    tensor_list = []
    for _ in size_list:
        tensor_list.append(torch.empty((max_size,), dtype=torch.uint8, device="cuda"))
    if local_size != max_size:
        padding = torch.empty(size=(max_size - local_size,), dtype=torch.uint8, device="cuda")
        tensor = torch.cat((tensor, padding), dim=0)
    dist.all_gather(tensor_list, tensor)

    data_list = []
    for size, tensor in zip(size_list, tensor_list):
        buffer = tensor.cpu().numpy().tobytes()[:size]
        data_list.append(pickle.loads(buffer))

    return data_list


def reduce_dict(input_dict, average=True):
    """
    Args:
        input_dict (dict): all the values will be reduced
        average (bool): whether to do average or sum
    Reduce the values in the dictionary from all processes so that all processes
    have the averaged results. Returns a dict with the same fields as
    input_dict, after reduction.
    """
    world_size = get_world_size()
    if world_size < 2:
        return input_dict
    with torch.no_grad():
        names = []
        values = []
        # sort the keys so that they are consistent across processes
        for k in sorted(input_dict.keys()):
            names.append(k)
            values.append(input_dict[k])
        values = torch.stack(values, dim=0)
        dist.all_reduce(values)
        if average:
            values /= world_size
        reduced_dict = {k: v for k, v in zip(names, values)}
    return reduced_dict


class MetricLogger(object):
    def __init__(self, delimiter="\t"):
        self.meters = defaultdict(SmoothedValue)
        self.delimiter = delimiter

    def update(self, **kwargs):
        for k, v in kwargs.items():
            if isinstance(v, torch.Tensor):
                v = v.item()
            assert isinstance(v, (float, int))
            self.meters[k].update(v)

    def __getattr__(self, attr):
        if attr in self.meters:
            return self.meters[attr]
        if attr in self.__dict__:
            return self.__dict__[attr]
        raise AttributeError("'{}' object has no attribute '{}'".format(
            type(self).__name__, attr))

    def __str__(self):
        loss_str = []
        for name, meter in self.meters.items():
            loss_str.append(
                "{}: {}".format(name, str(meter))
            )
        return self.delimiter.join(loss_str)

    def synchronize_between_processes(self):
        for meter in self.meters.values():
            meter.synchronize_between_processes()

    def add_meter(self, name, meter):
        self.meters[name] = meter

    def log_every(self, iterable, print_freq, header=None):
        i = 0
        if not header:
            header = ''
        start_time = time.time()
        end = time.time()
        iter_time = SmoothedValue(fmt='{avg:.4f}')
        data_time = SmoothedValue(fmt='{avg:.4f}')
        space_fmt = ':' + str(len(str(len(iterable)))) + 'd'
        if torch.cuda.is_available():
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}',
                'max mem: {memory:.0f}'
            ])
        else:
            log_msg = self.delimiter.join([
                header,
                '[{0' + space_fmt + '}/{1}]',
                'eta: {eta}',
                '{meters}',
                'time: {time}',
                'data: {data}'
            ])
        MB = 1024.0 * 1024.0
        for obj in iterable:
            data_time.update(time.time() - end)
            yield obj
            iter_time.update(time.time() - end)
            if i % print_freq == 0 or i == len(iterable) - 1:
                eta_seconds = iter_time.global_avg * (len(iterable) - i)
                eta_string = str(datetime.timedelta(seconds=int(eta_seconds)))
                if torch.cuda.is_available():
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time),
                        memory=torch.cuda.max_memory_allocated() / MB))
                else:
                    print(log_msg.format(
                        i, len(iterable), eta=eta_string,
                        meters=str(self),
                        time=str(iter_time), data=str(data_time)))
            i += 1
            end = time.time()
        total_time = time.time() - start_time
        total_time_str = str(datetime.timedelta(seconds=int(total_time)))
        print('{} Total time: {} ({:.4f} s / it)'.format(
            header, total_time_str, total_time / len(iterable)))


def collate_fn(batch):
    return tuple(zip(*batch))


def warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor):

    def f(x):
        if x >= warmup_iters:
            return 1
        alpha = float(x) / warmup_iters
        return warmup_factor * (1 - alpha) + alpha

    return torch.optim.lr_scheduler.LambdaLR(optimizer, f)


def mkdir(path):
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise


def setup_for_distributed(is_master):
    """
    This function disables printing when not in master process
    """
    import builtins as __builtin__
    builtin_print = __builtin__.print

    def print(*args, **kwargs):
        force = kwargs.pop('force', False)
        if is_master or force:
            builtin_print(*args, **kwargs)

    __builtin__.print = print


def is_dist_avail_and_initialized():
    if not dist.is_available():
        return False
    if not dist.is_initialized():
        return False
    return True


def get_world_size():
    if not is_dist_avail_and_initialized():
        return 1
    return dist.get_world_size()


def get_rank():
    if not is_dist_avail_and_initialized():
        return 0
    return dist.get_rank()


def is_main_process():
    return get_rank() == 0


def save_on_master(*args, **kwargs):
    if is_main_process():
        torch.save(*args, **kwargs)


def init_distributed_mode(args):
    if 'RANK' in os.environ and 'WORLD_SIZE' in os.environ:
        args.rank = int(os.environ["RANK"])
        args.world_size = int(os.environ['WORLD_SIZE'])
        args.gpu = int(os.environ['LOCAL_RANK'])
    elif 'SLURM_PROCID' in os.environ:
        args.rank = int(os.environ['SLURM_PROCID'])
        args.gpu = args.rank % torch.cuda.device_count()
    else:
        print('Not using distributed mode')
        args.distributed = False
        return

    args.distributed = True

    torch.cuda.set_device(args.gpu)
    args.dist_backend = 'nccl'
    print('| distributed init (rank {}): {}'.format(
        args.rank, args.dist_url), flush=True)
    torch.distributed.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                                         world_size=args.world_size, rank=args.rank)
    torch.distributed.barrier()
    setup_for_distributed(args.rank == 0)

In [ ]:
#forrás : https://github.com/matterport/Mask_RCNN

import math

def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:
        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



In [ ]:
num_epochs = 10
model.to(device)
    
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)


'''
for epoch in range(num_epochs):
    model.train()   
    i = 0
    loss_total = 0
    for imgs, annotations in train_data_loader:
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()   
        
        loss_total += loss_value
        i+=1
        
        if i % 50 == 0:
            print(f"Iteration #{i} loss: {loss_total/i}")

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()       
    print(f"Epoch #{epoch} loss: {loss_total/i}") 
'''
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_data_loader, device, epoch, print_freq=10)

In [ ]:
def apply_mask_train(image, mask, color, alpha=0.5):
    for c in range(3):
        for i in range(len(image[0])):
            for j in range(len(image[0][0])):
                if mask[j][i]==1:
                    image[c][i][j] = image[c][i][j] * (1 - alpha) + alpha * color[c] * 255
    return image

def apply_mask_test(image, mask, xmin, xmax, ymin, ymax, color, alpha=0.5):
    two_third_y = (ymax-ymin)*2/3 + ymin
    ymin = round(ymin.item())
    ymax = round(ymax.item())
    xmin = round(xmin.item())
    xmax = round(xmax.item())
    for c in range(3):
        for i in range(ymin, ymax):
            for j in range(xmin, xmax):
                if mask[i][j]<1e-6:
                    image[c][i][j] = image[c][i][j] * (1 - alpha) + alpha * color[c] * 255
    return image
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_image(img_tensor, annotation, test):    
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    # Display the image
    
    for i in range(len(annotation["boxes"])):
        box = annotation["boxes"][i]
        xmin, ymin, xmax, ymax = box

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
        
        label = annotation["labels"][i]
        
        print(label)
        toWrite = ""
        if label==1:
            toWrite += "With Mask"
        elif label==2:
            toWrite += "Without Mask"
        else:
            toWrite += "Incorrect Mask"
        
        ax.text(xmin, ymax+15, toWrite, fontsize=10, color="green")
        if(test):
            mask = annotation["masks"][i][0]
            img = apply_mask_test(img, mask, xmin, xmax, ymin, ymax, [255, 0, 0])
        else:
            mask = annotation["masks"][:, :, i]
            img = apply_mask_train(img, mask, [255, 0, 0])
        
    ax.imshow(img.permute(1,2,0))
    plt.show()
  
j = 0
for images, annotations in valid_data_loader:
    imgs = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    model.eval()
    with torch.no_grad():
        preds = model(imgs)    

    for i in range(len(imgs)):
        goodPred = { "boxes" : [], "labels" : [], "scores" : [], "masks" : []}
        for j in range(len(preds[i]["scores"])):
            if preds[i]["scores"][j] > 0.5:
                goodPred["boxes"].append(preds[i]["boxes"][j])
                goodPred["labels"].append(preds[i]["labels"][j])
                goodPred["scores"].append(preds[i]["scores"][j])
                goodPred["masks"].append(preds[i]["masks"][j])
        print("Prediction")
        plot_image(imgs[i], goodPred, True)
        
        '''print("Target")
        plot_image(imgs[i], targets[i], False)
        print(targets[i]["labels"])'''

In [ ]:
'''import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_image(img_tensor, annotation):    
    fig,ax = plt.subplots(1)
    img = img_tensor.cpu().data

    # Display the image
    ax.imshow(img.permute(1, 2, 0))
    
    for i in range(len(annotation["boxes"])):
        box = annotation["boxes"][i]
        xmin, ymin, xmax, ymax = box

        # Create a Rectangle patch
        rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')

        # Add the patch to the Axes
        ax.add_patch(rect)
        
        label = annotation["labels"][i]
        
        print(label)
        toWrite = ""
        if label==1:
            toWrite += "With Mask"
        elif label==2:
            toWrite += "Without Mask"
        else:
            toWrite += "Incorrect Mask"
        
        mask = annotation["masks"][i]
        ax.text(xmin, ymax+15, toWrite, fontsize=10, color="green")
        #img = apply_mask(img, mask, [124, 135, 242])
   
    plt.show()


j = 0
for images, annotations in valid_data_loader:
    imgs = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    model.eval()
    with torch.no_grad():
        preds = model(imgs)    

    for i in range(len(imgs)):
        for j in range(len(preds[i]["scores"])):
            goodPred = { "boxes" : [], "labels" : [], "scores" : [], "masks": []}
            if preds[i]["scores"][j] > 0.2:
                goodPred["boxes"].append(preds[i]["boxes"][j])
                goodPred["labels"].append(preds[i]["labels"][j])
                goodPred["scores"].append(preds[i]["scores"][j])
                goodPred["masks"].append(preds[i]["masks"][j])
        print("Prediction")
        plot_image(imgs[i], goodPred)'''